In [1]:
import numpy as np
from scipy import spatial
from scipy.linalg import eigh
import matplotlib.pyplot as plt
import pandas as pd
import re
from matplotlib import pyplot as plt


# Exercises week 11 - word embedding

Word embeddings are very popular representations for the vocabulary of any document corpus. It is one of the major breakthroughs of utilizing deep learning to solve challenging problems in natural language processing. A word embedding is a learned representation for the text, in which the words that have similar meaning have a similar representation. This meaning can be based varius aspects such as syntax, semantics and abstract relations between words.

For this exercise we use pre-trained word embedding from the GloVe (Global Vectors for Word Representation, https://nlp.stanford.edu/projects/glove/) project. In particular we use data trained on the socalled Wikipedia 2014 and Gigaword 5 corpus. This dataset contains $400000$ words, each represented by a feature vector (http://nlp.stanford.edu/data/glove.6B.zip). 

**Problems**
- Exercise 1 generate co-occurrence matrix for the dataset "Reviews.csv".
- Exercise 2 compares the word embeddings for different words and examine whether the word embedding captures various types of relations between the words.
- Exercise 3 uses principal component analysis to reduce the dimensionality of the word embedding.
- Exercise 4 uses transfer learning to address the problem of identifying analogies between words, utilizing the knowledge comprised in the word embedding.
- Exercise 5 illustrates whether the word embedding allows us to infer general conclusions about the original text corpus, by utilizing the methodology from the first exercises. In particular we examine wheter biases or stereotypes have been convayed to the word embedding.

# Exercise 1 (Optional) - Co-ocurance matrix



In [ ]:
data    = pd.read_csv("Reviews.csv")
reviews = data["Text"].values[:4] # take only first couple of reviews

## Find the distinct words

Hints:
- loop over all words in each review
- save only the distinct words in a list
- remember to convert all words to lower characters using ".lower()" on strings

In [ ]:
for review in reviews:
    ?

## Calculate $X_{ij}$
Hints:
- for each distinct word loop through all reviews again
- for each review find the location where that distinct word is using "np.where(words_in_review==distinct)[0]"
- take the previous $L=1$ and subsequent $L=1$ words of those locations and add 1 to the co-occurence matrix. You can use pandas DataFrame for indexing properly.
- optional: make your implementation handle $L$ to be an arbitrary positive integer

In [ ]:
X_ij        = ?


Plot $X$ as an image

In [ ]:
plt.imshow(X_ij.to_numpy())
plt.colorbar()


# Glove



**Loading the data set**

The following code loads the data and stores it in a *dictionary*. A dictionary is an abstract datastructure that stores key-value pairs. Given a key, the corresponding value can be stored or retrieved. The datastructure hence defines a list of keys and values and implements the functionality to link keys to values. For the dictionary declared in the code, the keys are the individual words (as strings) and the values are the associated GloVe feature vectors (as one dimensional numpy arrays).

$\star$ In the code, the data origines from a text file, where each line describes one word and its associated feature vector. Make sure you understand the string manipulation that is used to parse the data, within the for-loop in the code.

$\star$ When a feature vector is defined, the function np.asarray is given the argument "float32". Why is this necessary?

$\star$ What is the dimensionality of the data? 

In [3]:
filename = "glove.6B.50d.txt"

dictionary = {}
with open(filename,'r', encoding='utf-8') as file:
    for line in file:
        elements = line.split();
        word = elements[0];
        vector = np.asarray(elements[1:],"float32")
        dictionary[word] = vector;

## Exercise 2 - Word similarities



In this exercise you will inspect the word embedding for different words, in order to determine whether words that semantically have similar meaning (or are semantically related) also have similar representations under the embedding.

To compare two words, we use the cosine similarity score between the feature vectors representing the two words.
The following code implements two functions. *word_similarity* computes the similarity between two words while *nearest_neighbours* computes the $N$ nearest neighbours (most similar words) to a single word.

$\star$ Carefully examine the implementation of the two functions. Make sure you understand the code line by line.

The next cell calls the *word_similarity* function for different pairs of words withen various groups of words.

$\star$ Based on your common understanding, discuss the relation within the individual groups. One of the groups is called 'baseline'.

$\star$ Compare the similarity scores for the word pairs within each group. Do the relative differenses in scores reflect your own intuition? Does this inform you about certain relations captured by the word embedding? For instance, what can you learn from observing that 'Denmark' is more similar to 'Sweden' than to 'Copenhagen'?

$\star$ Instead of basing the similarity on the entire feature vector, try to use only the first five or ten values. Comment on whether you observe different outcomes.

$\star$ Select representative words from each group and use the *nearest_neighbours* function to find the respectively most similar words from the entire vocabulary. Try to examine words that you would expect to be both relatively similar and relatively different within the groups. Note why you intuitively would expect the words to be similar or different.
Comment on your findings and compare you results both within and between groups.

In [ ]:
def cos_sim(v1, v2):
    return 1 - spatial.distance.cosine(v1,v2);

def word_similarity(word1, word2):    
    v1 = dictionary[word1]
    v2 = dictionary[word2]
    similarity =  cos_sim(v1,v2)
    print("similarity score between",word1,"and",word2,"=",similarity)
    
def nearest_neighbours(word, num_neighbours):    
    words = np.asarray(list(dictionary.keys()))
    scores = np.zeros(len(words))    
    #find similarity score for all words in the dictionary 
    v = dictionary[word]
    for id in range(len(words)):
        word2 = words[id]
        v2 = dictionary[word2]
        scores[id] = cos_sim(v,v2)
    #sort scores and words according to score
    sorted_scores, sorted_words = zip(*sorted(zip(scores,words)))
    sorted_scores = sorted_scores[::-1]
    sorted_words = sorted_words[::-1]
    #print most similar words from the vocabulary
    print("most similar words to ",word)
    for x in range(num_neighbours):
        print(sorted_scores[x],"\t",sorted_words[x])    

In [ ]:
#humans and family structures
word_similarity("king","queen")
word_similarity("prince","princess")
word_similarity("prince","brother")
word_similarity("princess","sister")
word_similarity("prince","sister")
word_similarity("princess","brother")

word_similarity("brother","sister")
word_similarity("brother","son")
word_similarity("sister","daughter")
word_similarity("brother","daughter")
word_similarity("sister","son")

#prepositions and orientations
word_similarity("after","before")
word_similarity("over","under")
word_similarity("vertical","horizontal")
word_similarity("left","right")

#toponyms (places)
word_similarity("city","country")
word_similarity("city","town")
word_similarity("denmark","sweden")
word_similarity("denmark","copenhagen")
word_similarity("stockholm","copenhagen")
word_similarity("stockholm","denmark")
word_similarity("sweden","stockholm")
word_similarity("sweden","copenhagen")
word_similarity("walk","run")
word_similarity("run","sit")
word_similarity("walk","sit")

#Homonyms (words with different meanings)
word_similarity("second","minute")
word_similarity("first","second")
word_similarity("first","minute")



## Exercise 3 - Visualizing using PCA

In this exercise we use principal component analysis (PCA) to reduce the dimensionality of the feature vector space. This allows us to illustrate the spatial relation between different words from the vocabulary.

A function, *plot_pca* that conducts PCA on a sub-set of the data and projects these data-points onto the first two principal components is given below. The next cell then calls the *plot_pca* function for different lists of words. You are encuraged to modify the content of the lists if you expect that it may further support your findings in the following questions.

$\star$ Examine the code and observe how PCA is conducted.

$\star$ Comment on any linearity and distance between pairs of words.

$\star$ When visualizing for many words, comment on which words are grouped together. Does the grouping capture any semantic meaning?

$\star$ Evaluate whether you can use the PCA visualization to illustrate your findings from the previous exercise. Can your previous intuitions be supported by the plots?

In [ ]:
def plot_pca(word_list):
    
    #obtain dictionary of words from word_list
    sub_dictionary = {key:dictionary[key] for key in word_list}
    words = sub_dictionary.keys()
    vectors = np.asarray([sub_dictionary[key] for key in words]).T
    
    #pca on feature vectors for selected words
    mean_vector = np.mean(vectors,axis=1)
    data = vectors - mean_vector[:,None]
    #compute the covariance matrix
    S = np.cov(data)
    #obtain eigenvectors and eigenvalues
    eigenValues, eigenVectors = eigh(S)
    #sort according to size of eigenvalues
    eigenValues = eigenValues[::-1]
    eigenVectors = eigenVectors[:, ::-1]
    Y = eigenVectors.T@(data)
        
    #plot for first two principal components
    for label, x, y in zip(word_list, Y[0,:], Y[1,:]):
        plt.plot(x,y,"*")
        plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords='offset points')
    plt.show()

In [ ]:
plot_pca(["ship","harbor","car","garage"])
plot_pca(["father","mother","son","daughter"])
plot_pca(["king","queen","prince","princess","man","woman","boy","girl"])
plot_pca(["denmark","copenhagen","sweden","stockholm","germany","berlin","france","paris"])
plot_pca(["guitar","piano","saxophone","cat","dog","horse","tiger","book","article","journal","computer","telephone","dishwasher","gramaphone","house","garden","lawn"])

## Exercise 4 - Analogy solver


As defined by Aristotle as "an equality of proportions", an analogy can be considered a problem involving four terms, such that the second term is related to the first in a similar way as the forth term is related to the third.

An example of an analogy is;
- *evening* is to *morning* as *dinner* is to *breakfast*

Analogies are often used as verbal reasoning tests, where the test subject is asked to infer the similarity between two relations, by only being presented the first three terms of the analogy problem.
For instance, the subject is expected to infer the term *girl* from the problem;
- *brother* is to *sister* as *boy* is to *???*

These kind of problems are often included in assessments of intelligence (such as IQ or children development tests) as the ability to draw analogies are believed to reflect the underlying cognitive ability to represent and reason about higher-order relations.

In this exercise you are going to examine an AI algorithm for solving such analogy problems. As input, the algorithm is given the first three terms of an analogy problem. It will then rely on the linear substructure of the GloVe word embedding to find and return the best candidate word for the forth term.

**Algorithm:**
For a given problem, let $v_1,v_2,v_3$ be the vector representations for the first three terms. 
The guess for the forth term is found as the word for which the associated vector representation is most similar to the vector $v_d = (v_3 - (v_1-v_2))$. To compare vectors, the algorithm uses the cosine similarity measure.

$\star$ Argue why this approach is reasonable and why $v_d$ is computed as it is. Hint: It may be easier to build intuition from considering the problem in 2D.

$\star$ Test the algorithm with the provided examples as well as with analogy problems of your own. Investigate whether there are particular types of analogies that the algorithm cannot solve. Comment on why this is.

$\star$ Try to modify the algorithm so it returns the five 'best guesses'. Comment on whether the correct answer is among these guesses, in particular for the problems it could not solve before. 

$\star$ For real cognitive tests the analogy problem is often presented as a multiple-choise question. Modifying the algorithm to such a setting, would you expect it to perform better? 

In [ ]:
#algorithm for solving analogy problems
def find_analogy(w1, w2, w3):
    v1 = dictionary[w1];
    v2 = dictionary[w2];
    v3 = dictionary[w3];
    vd = v3 - (v1-v2);    
    max_similarity = -10000
    for word in dictionary.keys():
        #avoid guesses on already used terms
        if word!=w1 and word!=w2 and word!=w3:
            v = dictionary[word]
            similarity = 1 - spatial.distance.cosine(vd,v)
            if similarity > max_similarity:
                max_similarity = similarity;
                bestguess_word = word;
    return bestguess_word

In [ ]:
#examples of analogy problems

#w1 = "man"; w2 = "woman"; w3 = "king";
#w1 = "brother"; w2 = "sister"; w3 = "boy";
#w1 = "pen"; w2 = "paper"; w3 = "doctor";
#w1 = "denmark"; w2 = "copenhagen"; w3 = "germany";
#w1 = "denmark"; w2 = "copenhagen"; w3 = "france";
#w1 = "denmark"; w2 = "france"; w3 = "copenhagen";
#w1 = "car"; w2 = "parking"; w3 = "ship";
#w1 = "left"; w2 = "right"; w3 = "horizontal";
#w1 = "woman"; w2 = "man"; w3 = "mother";
#w1 = "man"; w2 = "woman"; w3 = "doctor";

w4 = find_analogy(w1,w2,w3)
print(w1,"is to",w2,"as",w3,"is to",w4)
plot_pca([w1,w2,w3,w4])


## Exercise 5 - Quantifying stereotypes

As for most machine learning algorithms, word embedding can be proned to biases. If the original data contained systematic biases, the learned representations may also reflect these biases. The GloVe machine learning algorithm was trained on Wikipedia texts. If these texts in general contained or described specific stereotypes, the word embedding will also capture these stereotypes. 
Often it is found that word embeddings contain gender stereotypes, often significantly with regards to professions.

$\star$ Use the plot_pca function to visualize the words "nurse", "doctor", "man" and "woman". Comment on the plot in terms of gender stereotypes. 

$\star$ Try to compare the cosine distance between various professions ("programmer", "secretary", "pilot", "president", "nurse",...) and the words "mother" and "farther". Comment on your findings.

$\star$ Evaluate on the terms that the word embedding relate to the two words "man" and "woman". For instance you can look at and compare the nearest neighbours to "man" and "woman" respectively. Are there any professions related to "man" and "woman"? Do "man" and "woman" have neighbours in common? 


$\star$ Discuss whether you can use the analogy solver to test for various biases or stereotypes that may have been convayed to the word embedding.

$\star$ Reflect on whether word embedding in general is a feasible approach to verify if a text corpus contains biases or stereotypes.
